---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="dbf31490" -->
# Table des matières
1. [Applications](#applications)
1. [Les forêts aléatoires aiment les complications!](#les-forêts-aléatoires-aiment-les-complications)
  1. [Le problème des discontinuités](#le-problème-des-discontinuités)
    1. [De quel type de discontinuités parle-t-on?](#de-quel-type-de-discontinuités-parle-t-on)
  1. [Le problème du grand nombre d'interactions](#le-problème-du-grand-nombre-dinteractions)
  1. [Le problème des ordres de grandeur](#le-problème-des-ordres-de-grandeur)
  1. [Le problème des variables hétéroclites](#le-problème-des-variables-hétéroclites)
1. [Voir l'arbre dans la forêt](#voir-larbre-dans-la-forêt)
  1. [Exemple 2-D](#exemple-2-d)
    1. [Comment utilise-t-on un tel arbre?](#comment-utilise-t-on-un-tel-arbre)
  1. [Principe de construction d'un arbre décisionnel en régression](#principe-de-construction-dun-arbre-décisionnel-en-régression)
    1. [Que se passerait-il s'il y avait un grand nombre de variables $x_i$?](#que-se-passerait-il-sil-y-avait-un-grand-nombre-de-variables-x_i)
  1. [Fragilité structurelle = variance des prédictions](#fragilité-structurelle-variance-des-prédictions)
  1. [Forces et faiblesses des arbres décisionnels en régression](#forces-et-faiblesses-des-arbres-décisionnels-en-régression)
1. [L'union fait la force: la forêt aléatoire](#lunion-fait-la-force-la-forêt-aléatoire)
  1. [Applications à l'analyse des séries temporelles complexes](#applications-à-lanalyse-des-séries-temporelles-complexes)
  1. [Forces et faiblesses des forêts aléatoires en régression](#forces-et-faiblesses-des-forêts-aléatoires-en-régression)
<!-- #endregion -->

<!-- #region id="35ecf3c0" -->
Comme on l'a vu dans les sections précédentes, la régression est une méthode d'apprentissage automatique
servant à prédire une réponse continue $y$ en fonction de plusieurs variables $X=\{x_{1}, \cdots, x_{N}\}$ selon

$$y=f(X, \Theta)$$

où $\Theta$ représente le vecteur des paramètres de la fonction $f$.

Beaucoup de problèmes de régression peuvent être abordés en utilisant les modèles linéaires dans les
coefficients (MLC) vus dans les modules précédents. Il
arrive toutefois que pour certains types de problèmes, leurs performances laissent à désirer. Dans
certains cas, c'est dû au non-respect des hypothèses sur lesquelles ces modèles sont basés.
Dans ces situations, la méthode de régression par forêt aléatoire a tendance à performer magistralement. Elle est
même souvent préférée aux réseaux de neurones pour cette raison.

Vous constaterez en lisant ce module une grande différence d'approches entre les deux méthodes:

- les MLC sont systématiques, prévisibles et utilisent les données d'entraînement en un bloc,
- les forêts aléatoires sont heuristiques, chaotiques et rebrassent constamment le jeu de données d'entraînement.
<!-- #endregion -->

<!-- #region id="7041a8f7" -->
# <a id=applications>Applications</a>
<!-- #endregion -->

<!-- #region id="ca392f5f" -->
Voici quelques applications et problèmes où les forêts aléatoires sont utilisées en régression:
<!-- #endregion -->

<!-- #region id="dea061f2" -->
#### Prévisions financières
<!-- #endregion -->

<!-- #region id="e33078fa" -->
- Prévoir l'évolution de l'inventaire de différents médicaments dans une pharmacie (Institut intelligence et données, IID).
- Prévoir l'évolution du taux de chômage en fonction de variables économiques locales, nationales et internationales (OCDE).
- Prédire le prix d'une maison dans un quartier en tenant compte de divers facteurs socio-économiques (Boston Housing Database).
- Prévoir la valeur de l'action d'une compagnie en fonction de diverses variables économiques.
<!-- #endregion -->

<!-- #region id="2acec25c" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/time-series-forecasting.png"  width="700" />
    <div>
    <font size="0.5">Image Source: https://www.datasciencecentral.com/profiles/blogs/time-series-forecasting-with-rnns</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="4196bcf4" -->
#### Médecine
<!-- #endregion -->

<!-- #region id="376df81e" -->
- Prédire l'évolution de la maladie d'un patient en fonction de ses paramètres physiologiques (Pima Indians Diabetes Database).
- Prédire l'évolution de la grippe saisonnière en fonction de statistiques médicales, météorologiques, etc. (OMS).
<!-- #endregion -->

<!-- #region id="5b4cb4ac" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/petri-dish.jpeg"  width="400" />
    <div>
    <font size="0.5">Image Source: https://www.flickr.com/photos/npmeijer/9696950969/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="9db97a22" -->
# <a id=les-forêts-aléatoires-aiment-les-complications>Les forêts aléatoires aiment les complications!</a>
<!-- #endregion -->

<!-- #region id="42e39a56" -->
Les forêts aléatoires font fi des problèmes suivants qui ne sont pas rares dans la pratique et qui posent de sérieux défis
aux modèles linéaires dans les coefficients.
<!-- #endregion -->

<!-- #region id="d1c0fd27" -->
## <a id=le-problème-des-discontinuités>Le problème des discontinuités</a>
<!-- #endregion -->

<!-- #region id="e9acf940" -->
Les MLC font l'hypothèse que la réponse $y$ varie de façon continue avec les
variables $x_{i}$ étudiés. Ils ne peuvent pas représenter des situations avec des discontinuités comme dans la figure
suivante qui ne contient qu'une seule variable $x$. Imaginez maintenant qu'il y a un grand nombre de variables
$x_{i}$ qui changent également de façon abrupte!
<!-- #endregion -->

<!-- #region id="3fde4883" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/function-discontinuity.gif"  width="300" />
    <div>
    <font size="0.5">Image Source: https://www.math10.com/en/algebra/functions/function-continuity/function-continuity.html/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="717a8045" -->
### <a id=de-quel-type-de-discontinuités-parle-t-on>De quel type de discontinuités parle-t-on?</a>
<!-- #endregion -->

<!-- #region id="1490fd3d" -->
Un exemple classique consiste à prédire le prix d'une maison dans une ville en tenant compte de plusieurs variables
$x_{i}$ tels que:

- la surface de la maison,
- le nombre de pièces de la maison,
- le salaire médian des résidents du quartier,
- le nombre d'écoles dans le quartier,
- le nombre de stations de métro dans le quartier,
- la distance à la station de métro la plus proche,
- la présence d'un centre d'achat à proximité.

Le prix d'une maison peut facilement augmenter de $10~\%$ si elle est située par exemple à moins de 200 m
d'une station de métro ou à moins d'un km d'une université, etc. Ces discontinuités ont un réel effet
sur les prédictions. Dans cet exemple, les MLC ne feraient pas de bonnes prédictions de prix de maisons.
<!-- #endregion -->

<!-- #region id="2af93529" -->
## <a id=le-problème-du-grand-nombre-dinteractions>Le problème du grand nombre d'interactions</a>
<!-- #endregion -->

<!-- #region id="f30c3df5" -->
Supposons que l'on ait une réponse $y$ qui dépende de N variables $x_{1}$ à $x_{N}$. On peut construire un
modèle simple, sans interaction, les reliant entre elles. Il contient N+1 termes et donc N+1 coefficients à estimer
à partir des données d'entraînement.

$$y = a_{0} + a_{1}x_{1} + \cdots + a_{N}x_{N}$$

Ajoutons maintenant les interactions entre les variables dans le modèle précédent

$$y = a_{0} + a_{1}x_{1} + \cdots + a_{N}x_{N} + a_{1,2}x_{1}x_{2} + \cdots + a_{N-1,N}x_{N-1}x_{N}$$

Il contient maintenant $N+1+N(N-1)/2$ termes et donc autant de coefficients à estimer à partir des
données d'entraînement. Le nombre d'interactions peut exploser lorsque le nombre de variables impliquées
est grand. Si on décide de toutes les inclure, il risque d'avoir plus de coefficients à estimer qu'il y a
de données d'entraînement! Si on décide plutôt de n'en utiliser que quelques-unes, lesquelles choisir?
<!-- #endregion -->

<!-- #region id="d8e6bec7" -->
## <a id=le-problème-des-ordres-de-grandeur>Le problème des ordres de grandeur</a>
<!-- #endregion -->

<!-- #region id="3a58e514" -->
Un autre problème est relié aux variables $x_{i}$ qui peuvent varier sur plusieurs ordres de grandeur, par
exemple pour une surface de terrain qui varie entre $200\, \text{m}^2$ et $1\,000\,000\, \text{m}^2$. On ne peut pas faire l'hypothèse
que la surface varie peu autour de sa valeur moyenne. C'est en se basant sur des approximations de ce genre que l'on
peut remplacer une fonction inconnue et complexe $y=f(X, \Theta)$ par des MLC. Les MLC ne sont plus valides dans ce cas.
<!-- #endregion -->

<!-- #region id="f450e2d9" -->
## <a id=le-problème-des-variables-hétéroclites>Le problème des variables hétéroclites</a>
<!-- #endregion -->

<!-- #region id="604c63ee" -->
Les MLC performent mal lorsque la réponse $y$ dépend de variables $x_{i}$ hétéroclites sans
lien entre elles. Probablement parce qu'il est impossible de modéliser analytiquement une fonction les contenant.

Tous ces problèmes sont traités efficacement avec les forêts aléatoires! Pourquoi alors continuer à utiliser
les MLC? Simplement parce qu'ils performent bien lorsque toutes les hypothèses
à la base de leur construction sont respectées. Cela arrive souvent.
<!-- #endregion -->

<!-- #region id="56cb58ca" -->
# <a id=voir-larbre-dans-la-forêt>Voir l'arbre dans la forêt</a>
<!-- #endregion -->

<!-- #region id="faaf863d" -->
Comme premier pas, intéressons nous d'abord à la méthode de régression par arbre décisionnel. Est elle
intimement liée à celle des forêts aléatoires que l'on verra dans un instant.

L'arbre décisionnel est un type de régression que l'on fait intuitivement lorsqu'on cherche
par exemple à estimer le temps de préparation d'un repas en tenant compte de quelques variables
(nombre de personnes, quantité d'ingrédients, etc.).
<!-- #endregion -->

<!-- #region id="bfe73600" -->
## <a id=exemple-2-d>Exemple 2-D</a>
<!-- #endregion -->

<!-- #region id="a95c5430" -->
La figure suivante
montre un exemple d'arbre décisionnel où l'on veut prédire une réponse $y$ en fonction de variables $x_{1}$ et $x_{2}$.
Les cases en vert sont les feuilles de l'arbre et celles en bleu sont les noeuds d'où les branches (les lignes) se séparent.
<!-- #endregion -->

<!-- #region id="ddc1165a" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/decision-tree-example.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://medium.com/pursuitnotes/decision-tree-regression-in-6-steps-with-python-1a1c5aa2ee16/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="48636e32" -->
La figure suivante montre la réponse $y=f(x_{1}, x_{2})$ prédite par le modèle. Les croix rouges
identifient les points utilisés pour entraînement celui-ci. Les lignes pointillées indiquent les frontières de décision
imposées par les branches (*split*) et délimitent différentes zones. Tous les points dans une même zone ont
la même réponse $y$.
<!-- #endregion -->

<!-- #region id="c3eb9f39" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/decision-tree-six-steps.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://medium.com/pursuitnotes/decision-tree-regression-in-6-steps-with-python-1a1c5aa2ee16/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="688de197" -->
### <a id=comment-utilise-t-on-un-tel-arbre>Comment utilise-t-on un tel arbre?</a>
<!-- #endregion -->

<!-- #region id="ea47a51e" -->
Il faut naviguer de haut en bas en suivant les décisions basées sur un ensemble de données $(x_1, x_2)$. Comme exemple, supposons
que l'on a $x_{1}=10$ et $x_{2}=100$. Au noeud du premier niveau, on teste la condition $x_1<20$?. La réponse est oui et on se dirige vers le noeud de gauche, au second niveau. À ce noeud, on teste ensuite la condition $x_2<200$?. La réponse est oui et on se dirige vers la feuille de gauche au niveau suivant. La réponse associée à la feuille est $y=300,5$. C'est la réponse prédite par l'arbre décisionnel, comme on veut le vérifier dans la figure précédente.

<!-- #endregion -->

<!-- #region id="2b7687aa" -->
## <a id=principe-de-construction-dun-arbre-décisionnel-en-régression>Principe de construction d'un arbre décisionnel en régression</a>
<!-- #endregion -->

<!-- #region id="d9eb1a31" -->
On va utiliser l'arbre précédent, à deux variables $x_{1}$ et $x_{2}$, pour expliquer comment un arbre
décisionnel est assemblé.

Au départ, il n'y a qu'une seule case verte contenant toutes les données d'entraînement $(x_1, x_2, y)$ puisqu'il n'y pas encore de noeud de décision. On sélectionne $x_{1}$ et on impose un seuil donné $\tau$. Toutes les valeurs d'entraînement $(x_1, x_2, y)$ pour lesquelles $x_1<\tau$ seront dirigées vers la feuille de gauche, et les autres vers la feuille de droite. On calcule la variance des valeurs d'entraînement $\sigma^2(y)$ dans chaque feuille, puis on calcule la somme des deux variances. Il faut ensuite
déterminer pour quelle valeur de seuil $\tau$ cette somme des variances est minimale. On sépare enfin des données avec cette valeur. On se retrouve alors avec deux ensembles de données, un par feuille,
qui ont des moyennes $\mu(y)$ différentes et sont relativement uniformes. On refait ensuite le même exemple
avec toutes les valeurs d'entraînement, mais en utilisant, cette fois-ci, la variable $x_2$. Après cette étape, on dispose de deux sommes de variances, une pour $x_1$ et une pour $x_2$. On détermine laquelle des deux variables a la plus faible somme. Dans le cas de l'arbre décisionnel ci-dessus, $x_{1}$ a gagné et son seuil optimal était de 20.

On applique à nouveau la même procédure aux données $(x_{1}, x_{2}, y)$ dans la feuille de gauche et ensuite avec
les données dans la feuille de droite. On se retrouve alors avec une branche gauche ayant sélectionné
$x_{2}$ avec un seuil de 200 et une branche droite ayant sélectionné à nouveau $x_{2}$ avec un seuil
de 170. L'idée est de continuer ainsi de façon itérative feuille après feuille, tout en respectant la structure
de l'arbre. À la fin, la valeur moyenne $\mu(y)$ dans chaque feuille sera la réponse $y$ prédite pour
les futures données $(x_{1}, x_{2})$ qui y aboutiront après avoir traversé l'arbre.

La structure de l'arbre est déterminée entre autres par le nombre minimum de données d'entraînement qui peut
se retrouver dans chaque feuille, et le nombre de niveaux d'embranchements (3 dans la figure du haut). Plus sa structure
est complexe, plus grand est le nombre de feuilles et donc le nombre de réponses $y$ prédites, et plus complexe est
la fonction modélisée. Toutefois, trop de feuilles et d'embranchements font apparaître le
phénomène du surapprentissage dont il sera question dans le module correspondant sur la méthodologie.
<!-- #endregion -->

<!-- #region id="61cae21a" -->
### <a id=que-se-passerait-il-sil-y-avait-un-grand-nombre-de-variables-x_i>Que se passerait-il s'il y avait un grand nombre de variables $x_i$?</a>
<!-- #endregion -->

<!-- #region id="5d271873" -->
Évidemment, on ne peut pas toutes les tester à chaque noeud de l'arbre; ça prendrait trop de temps. Dans ce cas, on
sélectionne un nombre prédéterminé de variables à utiliser à chaque noeud. Par exemple, si on avait 100 variables
$x_i$, on pourrait n'en utiliser que cinq à chaque noeud; elles seraient sélectionnées aléatoirement parmi les 100. C'est un des multiples paramètres des arbres décisionnels.

<!-- #endregion -->

<!-- #region id="a41c3c21" -->
## <a id=fragilité-structurelle-variance-des-prédictions>Fragilité structurelle = variance des prédictions</a>
<!-- #endregion -->

<!-- #region id="6eefb3d9" -->
On peut construire des arbres de régression avec beaucoup de variables $x_{i}$.
Les arbres deviennent toutefois imprécis et fragiles dans leurs dernières branches. Par exemple, si l'on
ajoute ou soustrait quelques données à l'ensemble d'entraînement d'un arbre, sa structure finale
peut être assez différente d'un cas à l'autre. Ainsi, les arbres décisionnels sont fragiles, car
de petites variations dans les données changent leurs structures. Il en résulte une grande
variabilité dans leurs prédictions.

La librairie
__[Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)__
contient une fonction pour effectuer de la régression par arbre de décision.
<!-- #endregion -->

<!-- #region id="5e30155f" -->
## <a id=forces-et-faiblesses-des-arbres-décisionnels-en-régression>Forces et faiblesses des arbres décisionnels en régression</a>
<!-- #endregion -->

<!-- #region id="32a1728c" -->
#### Forces
<!-- #endregion -->

<!-- #region id="56a63039" -->
- ils permettent l'identification des variables $x_{i}$ importantes,
- celles-ci sont les plus utilisées pour prendre des décisions à travers un arbre,
- ils ont une grande interprétabilité; leurs décisions sont faciles à expliquer.
<!-- #endregion -->

<!-- #region id="cbc4e26d" -->
#### Faiblesses
<!-- #endregion -->

<!-- #region id="7f943a3d" -->
- ils ne prédisent la réponse $y$ que dans le domaine d'entraînement délimité par les variables $x_{i}$, 
- l'extrapolation de la réponse $y$ hors de ce domaine est mauvaise,
- ils ont une variabilité importante dans leurs prédictions.
<!-- #endregion -->

<!-- #region id="db481149" -->
# <a id=lunion-fait-la-force-la-forêt-aléatoire>L'union fait la force: la forêt aléatoire</a>
<!-- #endregion -->

<!-- #region id="12a7c299" -->
Les arbres décisionnels, pris individuellement, ne sont pas précis, mais la combinaison d'un grand nombre d'entre eux
permet d'obtenir des prédictions beaucoup plus précises.

Cette idée est à la base des méthodes d'ensemble qui se divisent en deux grandes approches ou stratégies;
le *Bagging* et le *Boosting*. Elles sont utilisées en régression ainsi qu'en classification.
Chacune consiste à combiner des régresseurs (ou des classificateurs) peu performants afin que leurs forces
s'additionnent et leurs faiblesses se soustraient mutuellement. Dans ce qui suit, nous n'allons
discuter que du *Bagging* et de sa méthode la plus connue, les forêts aléatoires.

La figure suivante montre un exemple de forêt aléatoire combinant 600 arbres décisionnels
<!-- #endregion -->

<!-- #region id="55b7b6e5" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/random-forest-example.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://www.keboola.com/blog/random-forest-regression/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="c3f9212d" -->
Chaque arbre n'utilise qu'un sous-ensemble des variables $x_{i}$ disponibles; ses performances seront
limitées pour cette raison. C'est un régresseur peu performant (*weak learner*).
Ses prédictions moyennes sont passables en plus d'avoir une grande variabilité.

Chaque arbre utilise également le même nombre de données d'entraînement, mais les jeux d'entraînement sont tous
différents entre eux. Il sont générés avec la méthode
__[*bootstrap*](https://fr.wikipedia.org/wiki/Bootstrap_(statistiques))__. Un bootstrap consiste à
créer des « nouveaux échantillons » statistiques, mais uniquement par tirage
avec remise, à partir de l'échantillon initial.
Cette approche permet de simuler un plus grand nombre de jeux de données tout en respectant la distribution
statistique du jeu original. Chaque nouveau jeu de données peut contenir des
données dupliquées et manquer certaines des données originales. De plus, les variables $x_{i}$ sont
différents d'un jeu à l'autre, mais leur nombre demeure le même. Par exemple, si le jeu original d'entraînement contenait
1 000 données $(X,y)$ avec 20 variables, soit $X=\{x_{1},\cdots, x_{20}\}$, chaque nouveau jeu d'entraînement contiendra
1 000 données $(X',y)$ où $X'$ contiendra 10 variables $x_{i}$ choisis aléatoirement parmi les 20.

Si l'on construisait un arbre décisionnel (AD) unique contenant l'ensemble des données d'entraînement $(X,y)$, la
réponse serait
$$y = AD(X,\Theta)$$
avec les paramètres $\Theta$ définissant l'arbre. Dans l'exemple de forêt aléatoire ci-dessus, on rassemble
plutôt 600 arbres décisionnels utilisant chacun un jeu différent de données d'entraînement $(X',y')$. On
calcule ensuite la moyenne des 600 réponses prédites

$$y = \frac{1}{600}\sum_{i=1}^{600} AD(X',\Theta_{i})$$
avec les paramètres $\Theta_{i}$ définissant chaque arbre.

La librairie __[Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)__
contient une fonction  pour effectuer de la régression par forêt aléatoire. Plusieurs paramètres peuvent être ajustés tel que le nombre d'arbres,
le nombre de variables $x_{i}$ à utiliser, le nombre de niveaux maximal de chaque arbre, etc.
<!-- #endregion -->

<!-- #region id="942bff6c" -->
## <a id=applications-à-lanalyse-des-séries-temporelles-complexes>Applications à l'analyse des séries temporelles complexes</a>
<!-- #endregion -->

<!-- #region id="29501adc" -->
La régression par forêt aléatoire peut être appliquée à l'analyse de séries temporelles. De façon générale, on a

$$y = f(x_1,\cdots, x_N, \Theta)$$

Si l'on remplace les variables $x_i$ par leurs équivalents temporels $x_i(t)$, l'équation devient

$$y(t) = f(x_1(t),\cdots, x_N(t), \Theta)$$

Les forêts aléatoires, comme les arbres décisionnels, ne tiennent pas compte de l'ordre des données. Ainsi, de leur point de vue, une série temporelle n'est qu'une séquence de nombres. On peut la réarranger aléatoirement et ça ne changera rien à l'entraînement. À chaque donnée de type $(x_1(t),\cdots, x_N(t), y(t))$ correspond un échantillon de type $(x_1^{(i)},\cdots, x_N^{(i)}, y^{(i)})$. L'indice $i$ représente le numéro de l'échantillon tel que $1\le i \le n$, où $n$ est le nombre de données. L'ordre temporel des données n'a pas à être respecté puisque les données sont traitées de façon indépendante.

Cette approche permet de modéliser des comportements complexes en fonction d'une grande quantité de variables temporelles $x_{i}(t)$. On utilise ainsi les forêts aléatoires pour modéliser l'évolution des ventes et des profits de compagnies. La
seconde figure du module montre un exemple en modélisation boursière/financière.
<!-- #endregion -->

<!-- #region id="250d6938" -->
## <a id=forces-et-faiblesses-des-forêts-aléatoires-en-régression>Forces et faiblesses des forêts aléatoires en régression</a>
<!-- #endregion -->

<!-- #region id="e04e8dc5" -->
#### Forces
<!-- #endregion -->

<!-- #region id="620f5fc4" -->
- elles permettent l'identification des variables $x_{i}$ importantes,
- elles prennent en compte les interactions entre les variables sans avoir à les spécifier à l'avance,
- elles sont robustes au problème de corrélation entre les variables, 
- elles permettent de traiter des séries temporelles de données. 
<!-- #endregion -->

<!-- #region id="65bd4332" -->
#### Faiblesses
<!-- #endregion -->

<!-- #region id="9260670d" -->
- elles ne prédisent la réponse $y$ que dans le domaine d'entraînement délimité
par les variables $x_{i}$, 
- l'extrapolation de la réponse $y$ hors de ce domaine est mauvaise;
ça explique pourquoi les principales applications des forêts aléatoires sont en classification plutôt qu'en régression, 
- leurs prédictions sont difficiles à expliquer dû à l'aspect heuristique de leur construction, 
- elles ont de la difficulté à traiter les données clairsemées (*sparse data*) c.-à-d. lors la matrice des variables $x$ contient beaucoup de 0 et peu de 1.
<!-- #endregion -->
